In [14]:
import subprocess
import threading

#istallazione di ollama
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
######################################################################## 100.0%#=#=#                                                                          
>>> Installing ollama to /usr/local/bin...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> NVIDIA GPU installed.
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [15]:
def start_ollama():
    t = threading.Thread(target=lambda: subprocess.run(["ollama", "serve"]),daemon=True)
    t.start()

In [16]:
def pull_model(local_llm):
    !ollama pull local_llm

In [17]:
def start_model(local_llm):        
    t2 = threading.Thread(target=lambda: subprocess.run(["ollama", "run", local_llm]),daemon=True)
    t2.start()

In [18]:
%%capture --no-stderr
%pip install -U scikit-learn==1.3 langchain-ai21 langchain_community tiktoken langchainhub langchain langgraph

In [19]:
import os

os.environ["LANGCHAIN_API_KEY"] = "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"
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]="hate-speech-project"

In [20]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tqdm import tqdm
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

In [21]:
# todo #all'incirca 7.000 testi di test

df = pd.read_csv("/kaggle/input/hate-speech-detection-curated-dataset/HateSpeechDatasetBalanced.csv")

print(len(df))

#label_0 = df[df["Label"]==0] #no hate speech
#label_1 = df[df["Label"]==1] #hate speech
#print(f"Label con valore 0: {len(label_0)}")
#print(f"Label con valore 1: {len(label_1)}")

# Campiona l'1% del DataFrame
sampled_df = df.sample(frac=0.01, random_state=42)

x_test = sampled_df["Content"]
y_test = sampled_df["Label"]

print(len(x_test))
print(len(y_test))

label_0 = sampled_df[sampled_df["Label"]==0] #no hate speech
label_1 = sampled_df[sampled_df["Label"]==1] #hate speech
print(f"Label con valore 0: {len(label_0)}")
print(f"Label con valore 1: {len(label_1)}")

726119
7261
7261
Label con valore 0: 3584
Label con valore 1: 3677


In [22]:
examples = [
    {"input": "We need to keep our culture pure.", "output": "yes"},
    {"input": "Fuck you.", "output": "yes"},
    {"input": "It’s important to respect other people's choices.", "output": "no"},
    {"input": "Some groups of people are bastards", "output": "yes"},
    {"input": "I think the weather is nice today.", "output": "no"},
    {"input": "Go back to where you came from, asshole.", "output": "yes"},
    {"input": "Have a great day!.", "output": "no"},
    {"input": "They're just different, and that's not a bad thing, but they shouldn't try to change us.", "output": "yes"},
]

In [23]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

def create_prompt(shot):
    example_prompt  = ChatPromptTemplate.from_messages(
        [
            ("user", "{input}"),
            ("assistant", "{output}"),
        ]
    )

    few_shot_prompt = FewShotChatMessagePromptTemplate(
        example_prompt=example_prompt ,
        examples=examples[:shot], #:2 two-shot, #4: four-shot #8 eight-shot
    )
    
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", """You are an helpful assistant who has to detect the presence of hate speech.
                Hate speech is speech that attacks a person or group based on attributes such as race, religion, ethnic origin, national origin, sex, disability, sexual orientation, or gender identity. 
                You have to answer "yes" if it contains hate speech, or "no" if it doesn't contain hate speech. 
                Give only the answer "yes" or "no", NO PREAMBLE, NO EXPLANATIONS."""),
             few_shot_prompt, 
            ("user", "{input}"),
        ] if shot>0 else
        [
            ("system", """You are an helpful assistant who has to detect the presence of hate speech.
                Hate speech is speech that attacks a person or group based on attributes such as race, religion, ethnic origin, national origin, sex, disability, sexual orientation, or gender identity. 
                You have to answer "yes" if it contains hate speech, or "no" if it doesn't contain hate speech. 
                Give only the answer "yes" or "no", NO PREAMBLE, NO EXPLANATIONS."""),
            ("user", "{input}"),
        ]
    )
    return prompt

In [24]:
#import functools
import sys
import io

def hate_speech_detection(llm, shot):
    prompt_final = create_prompt(shot)
    hate_speech_detection = prompt_final | llm
    return hate_speech_detection

In [25]:
def predict(llm,x_test,shot):
    y_pred = []
    chain = hate_speech_detection(llm,shot)
    for x in tqdm(x_test):
        answer = chain.invoke({"input": x})
        #print(answer)
        if "no" in answer.lower(): y_pred.append(0)
        else: y_pred.append(1)
    return y_pred

In [ ]:
import time
import json

models = ["mistral"]
shots = [2,4,8]

def write_file(filename,content):
    with open(filename, 'w') as file:
        json.dump(content, file, indent=4)

for model in models:
    start_ollama()
    pull_model(model)
    start_model(model)
    time.sleep(500)
    llm = Ollama(model=model, temperature=0)
    for shot in shots:
        y_pred = predict(llm,x_test,shot)
        if model=="llama3.1": 
            write_file(f"/kaggle/working/prediction_hate_speech_llama31_{shot}_shot.json", y_pred)
        else:
            write_file(f"/kaggle/working/prediction_hate_speech_{model}_{shot}_shot.json", y_pred)

2024/08/13 11:13:28 routes.go:1123: INFO server config env="map[CUDA_VISIBLE_DEVICES: GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_LLM_LIBRARY: OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/root/.ollama/models OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://*] OLLAMA_RUNNERS_DIR: OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: ROCR_VISIBLE_DEVICES:]"
time=2024-08-13T11:13:28.401Z level=INFO source=images.go:782 msg="total blobs: 5"
time=2024-08-13T11:13:28.402Z level=INFO source=images.go:790 msg="total unused blobs removed: 0"
time=2024-08-13T11:13:28.

[GIN] 2024/08/13 - 11:13:35 | 200 |      50.231µs |       127.0.0.1 | HEAD     "/"


time=2024-08-13T11:13:35.041Z level=INFO source=payload.go:44 msg="Dynamic LLM libraries [cpu cpu_avx cpu_avx2 cuda_v11 rocm_v60102]"
time=2024-08-13T11:13:35.041Z level=INFO source=gpu.go:204 msg="looking for compatible GPUs"
time=2024-08-13T11:13:35.166Z level=INFO source=types.go:105 msg="inference compute" id=GPU-7d20e003-7065-4290-44bd-58268e47a8c4 library=cuda compute=6.0 driver=12.4 name="Tesla P100-PCIE-16GB" total="15.9 GiB" available="15.6 GiB"


pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ [GIN] 2024/08/13 - 11:13:35 | 200 |  401.999801ms |       127.0.0.1 | POST     "/api/pull"
pulling manifest ⠸ pulling manifest 
Error: pull model manifest: file does not exist
[GIN] 2024/08/13 - 11:13:35 | 200 |      33.127µs |       127.0.0.1 | HEAD     "/"
[GIN] 2024/08/13 - 11:13:35 | 200 |    10.08448ms |       127.0.0.1 | POST     "/api/show"
INFO [main] build info | build=1 commit="1e6f655" tid="134057973067776" timestamp=1723547615
INFO [main] system info | n_threads=2 n_threads_batch=-1 system_info="AVX = 1 | AVX_VNNI = 0 | AVX2 = 0 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 0 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 0 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | " tid="134057973067776" timestamp=1723547615 total_threads=4
INFO [main] HTTP server listening | hostname="127.0.0.1" n_threads_http="6" port="32969" tid="13405797

⠙ time=2024-08-13T11:13:35.809Z level=INFO source=sched.go:710 msg="new model will fit in available VRAM in single GPU, loading" model=/root/.ollama/models/blobs/sha256-ff82381e2bea77d91c1b824c7afb83f6fb73e9f7de9dda631bcdbca564aa5435 gpu=GPU-7d20e003-7065-4290-44bd-58268e47a8c4 parallel=4 available=16790978560 required="5.9 GiB"
time=2024-08-13T11:13:35.809Z level=INFO source=memory.go:309 msg="offload to cuda" layers.requested=-1 layers.model=33 layers.offload=33 layers.split="" memory.available="[15.6 GiB]" memory.required.full="5.9 GiB" memory.required.partial="5.9 GiB" memory.required.kv="1.0 GiB" memory.required.allocations="[5.9 GiB]" memory.weights.total="4.7 GiB" memory.weights.repeating="4.6 GiB" memory.weights.nonrepeating="105.0 MiB" memory.graph.full="560.0 MiB" memory.graph.partial="585.0 MiB"
time=2024-08-13T11:13:35.811Z level=INFO source=server.go:393 msg="starting llama server" cmd="/tmp/ollama1185196250/runners/cuda_v11/ollama_llama_server --model /root/.ollama/models

...

[GIN] 2024/08/13 - 12:14:08 | 200 |  215.005146ms |       127.0.0.1 | POST     "/api/generate"


 29%|██▉       | 2139/7261 [52:13<49:27,  1.73it/s]

[GIN] 2024/08/13 - 12:14:08 | 200 |   642.33925ms |       127.0.0.1 | POST     "/api/generate"


 29%|██▉       | 2140/7261 [52:13<56:15,  1.52it/s]

[GIN] 2024/08/13 - 12:14:09 | 200 |  826.864621ms |       127.0.0.1 | POST     "/api/generate"


In [ ]:
import json

def load_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

#y_pred = load_json("")

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred, labels=[0,1], output_dict=True)

print(f"Accuracy: {accuracy}")
print(f"Confusion Matrix:\n{conf_matrix}")
print(f"Classification Report:\n{class_report}")

In [ ]:
# Salva risultati

import json

def load_json(filename):
    with open(filename, 'r') as file:
        return json.load(file)

def write_file(filename,content):
    with open(filename, 'w') as file:
        json.dump(content, file, indent=4)
        

dict_to_write = {"class_report":class_report, "0_0":int(conf_matrix[0][0]), "0_1":int(conf_matrix[0][1]), "1_0":int(conf_matrix[1][0]), "1_1":int(conf_matrix[1][1])}        
write_file("/kaggle/working/test_hate_speech_mistral_zero_shot.json", dict_to_write)